<a href="https://colab.research.google.com/github/nspgav/AFI-project/blob/master/Getting_started_with_BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Before you begin


1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.


### Provide your credentials to the runtime

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Optional: Enable data table display

Colab includes the ``google.colab.data_table`` package that can be used to display large pandas dataframes as an interactive data table.
It can be enabled with:

In [ ]:
%load_ext google.colab.data_table

If you would prefer to return to the classic Pandas dataframe display, you can disable this by running:
```python
%unload_ext google.colab.data_table
```

# Use BigQuery via magics

The `google.cloud.bigquery` library also includes a magic command which runs a query and either displays the result or saves it to a variable as a `DataFrame`.

In [ ]:
# Display query output immediately

%%bigquery --project ne-fprt-data-cloud-production
SELECT 
  COUNT(*) as total_rows
FROM `fp_ia.fp_ia_chot_doc_main`

,total_rows
0,326


In [ ]:
# Save output in a variable `df`

%%bigquery --project yourprojectid df
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

In [ ]:
df

,total_rows
0,114420316


# Use BigQuery through google-cloud-bigquery

See [BigQuery documentation](https://cloud.google.com/bigquery/docs) and [library reference documentation](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html).

The [GSOD sample table](https://bigquery.cloud.google.com/table/bigquery-public-data:samples.gsod) contains weather information collected by NOAA, such as precipitation amounts and wind speeds from late 1929 to early 2010.


### Declare the Cloud project ID which will be used throughout this notebook

In [ ]:
project_id = 'ne-fprt-data-cloud-production'

### Sample approximately 2000 random rows

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

sample_count = 100
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `fp_ia.fp_ia_chot_doc_main`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `fp_ia.fp_ia_chot_doc_main`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count)

Full dataset has 326 rows


### Describe the sampled data

In [ ]:
df.describe()

,transaction_num,pos_number
count,105.000000,105.000000
mean,5352.276190,3.838095
std,2964.722268,5.486213
min,10.000000,1.000000
25%,2831.000000,1.000000
50%,6479.000000,2.000000
75%,8087.000000,4.000000
max,9776.000000,29.000000


### View the first 10 rows

In [ ]:
df.head(10)

,doc_ref,business_date,transaction_date,transaction_num,store_id,store_name,cashier_emp_id,cashier_name,pos_number,purchase_amt,doc_status,finding,action_taken,remarks,ia_remarks
0,CHOT-120620-431-2250,2020-06-12,2020-06-12 17:16:45+00:00,2250,431,Compassvale Link,21989,NG SIEW HONG,2,-7.95,Open,None,None,None,None
1,CHOT-230520-363-6485,2020-05-23,2020-05-23 07:01:23+00:00,6485,363,Taman Jurong,1060432,GOH WEI ZHAO,4,33.7,Open,None,None,None,None
2,CHOT-220520-363-9139,2020-05-22,2020-05-22 10:51:27+00:00,9139,363,Taman Jurong,1060432,GOH WEI ZHAO,1,46.82,Open,None,None,None,None
3,CHOT-230520-363-6515,2020-05-23,2020-05-23 07:43:28+00:00,6515,363,Taman Jurong,1060432,GOH WEI ZHAO,4,55.25,Open,None,None,None,None
4,CHOT-220520-363-9167,2020-05-22,2020-05-22 11:59:46+00:00,9167,363,Taman Jurong,1060432,GOH WEI ZHAO,1,114.4,Open,None,None,None,None
5,CHOT-230520-363-6561,2020-05-23,2020-05-23 08:53:16+00:00,6561,363,Taman Jurong,1060432,GOH WEI ZHAO,4,39.75,Open,None,None,None,None
6,CHOT-230520-363-6510,2020-05-23,2020-05-23 07:32:12+00:00,6510,363,Taman Jurong,1060432,GOH WEI ZHAO,4,8.3,Open,None,None,None,None
7,CHOT-220520-363-9121,2020-05-22,2020-05-22 10:25:57+00:00,9121,363,Taman Jurong,1060432,GOH WEI ZHAO,1,41.39,Open,None,None,None,None
8,CHOT-220520-363-9075,2020-05-22,2020-05-22 09:16:33+00:00,9075,363,Taman Jurong,1060432,GOH WEI ZHAO,1,41.4,Open,None,None,None,None
9,CHOT-220520-363-9147,2020-05-22,2020-05-22 11:02:08+00:00,9147,363,Taman Jurong,1060432,GOH WEI ZHAO,1,60.35,Open,None,None,None,None


In [ ]:
# 10 highest total_precipitation samples
df.sort_values('total_precipitation', ascending=False).head(10)[['station_number', 'year', 'month', 'day', 'total_precipitation']]

,station_number,year,month,day,total_precipitation
644,230220,1964,7,15,5.91
1155,985430,2008,12,8,3.46
1196,248260,1961,11,1,2.95
1588,257670,1959,8,9,2.95
980,299150,1962,3,1,2.95
1325,470250,1965,11,25,2.95
1917,288380,1994,8,6,2.32
1211,585190,1995,4,14,2.32
250,647000,2005,8,19,2.20
1418,964710,1975,9,8,1.97


# Use BigQuery through pandas-gbq

The `pandas-gbq` library is a community led project by the pandas community. It covers basic functionality, such as writing a DataFrame to BigQuery and running a query, but as a third-party library it may not handle all BigQuery features or use cases.

[Pandas GBQ Documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_gbq.html)

In [ ]:
import pandas as pd

df = pd.io.gbq.read_gbq('''
  SELECT *  FROM `fp_ia.fp_ia_chot_doc_main`
''', project_id=project_id, dialect='standard')

df.head() #print top 5

,doc_ref,business_date,transaction_date,transaction_num,store_id,store_name,cashier_emp_id,cashier_name,pos_number,purchase_amt,doc_status,finding,action_taken,remarks,ia_remarks
0,CHOT-120620-431-2211,2020-06-12,2020-06-12 16:06:08+00:00,2211,431,Compassvale Link,21989,NG SIEW HONG,2,7.95,Open,None,None,None,None
1,CHOT-120620-431-2250,2020-06-12,2020-06-12 17:16:45+00:00,2250,431,Compassvale Link,21989,NG SIEW HONG,2,-7.95,Open,None,None,None,None
2,CHOT-230520-363-6489,2020-05-23,2020-05-23 07:06:17+00:00,6489,363,Taman Jurong,1060432,GOH WEI ZHAO,4,16,Open,None,None,None,None
3,CHOT-220520-363-9058,2020-05-22,2020-05-22 08:52:56+00:00,9058,363,Taman Jurong,1060432,GOH WEI ZHAO,1,79.7,Open,None,None,None,None
4,CHOT-220520-363-9146,2020-05-22,2020-05-22 10:59:51+00:00,9146,363,Taman Jurong,1060432,GOH WEI ZHAO,1,24.8,Open,None,None,None,None
